In [1]:
import pyorient

In [2]:
print client.db_size()

NameError: name 'client' is not defined

In [6]:
print client.db_size()

12450


In [6]:
1/1.0

1.0

In [2]:
#note: cannot drop databsae if it is open
client.db_drop("Robots")

NameError: name 'client' is not defined

In [2]:
#step 1: connect to database
client = pyorient.OrientDB("localhost", 2424)
client.connect("root", "admin")

47

In [26]:
#step 2: create database if it not exists (create database before running tests)
#client.db_create("Robots", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)

In [3]:
#step 3: Open the database 
client.db_open("Robots", "root", "admin")

In [163]:
#step 3: Init required vertexes:
verteces = ['Root','LocationEvent']
for vertex in verteces:  
    try:
        client.command("create class "+vertex+" extends V") 
    except:
        pass

In [132]:
#create Root vertex
create_vertex("Root","robot_id=1")

{'@Root':{'robot_id': 1},'version':1,'rid':'#20:0'}


In [203]:
client.command("insert into " + vertex + " set data='dummy'")[0]._rid

'#24:0'

In [205]:
verteces = ['LocationEvent','HandleBarVoltageEvent',
'MototBarVoltageEvent','PoseEvent','RGBEvent']
for i in range(len(verteces)):
    print verteces[i]
    

LocationEvent
HandleBarVoltageEvent
MototBarVoltageEvent
PoseEvent
RGBEvent


In [129]:
#step 4: Init required Edges:
edges = ['Event_at']
for edge in edges:  
    try:
        client.command("create class "+edge+" extends E") 
    except:
        pass

In [150]:
import datetime
#step 5: generate sample location events and attach it to root vertex
def create_event(vertex,data):
    event_rid = client.record_create( 22, { vertex :  data  } )._rid
    print event_rid
    
    client.command(
    "create edge Event_at from ("
    "select * from #18:0"
    ") to ("
    "select * from " + event_rid +
    ")"
    )
    
for i in range(5):
    create_event('@LocationEvent',{'datetidme': datetime.datetime.now()})

#22:37
#22:38
#22:39
#22:40
#22:41


In [5]:
import datetime
current_time = datetime.datetime.now()
start_time_range = current_time - datetime.timedelta(seconds=10)
end_time_range = current_time
print start_time_range,end_time_range

today_beginning = datetime.datetime.combine(datetime.date.today(), datetime.time())
print today_beginning.strftime('%Y-%m-%d %H:%M:%S')

2017-12-03 15:50:10.845357 2017-12-03 15:50:20.845357
2017-12-03 00:00:00


In [67]:
def get_query(query_id):
    robot_id = "1"
    if query_id is 0:
        current_time = datetime.datetime.now()
        start_time_range = (current_time - datetime.timedelta(seconds=10)).strftime('%Y-%m-%d %H:%M:%S')
        end_time_range = current_time.strftime('%Y-%m-%d %H:%M:%S')
        query = "select  image_base64,blob,timestamp,@class from (select expand( out( Event_in )) from Root where robot_id="+robot_id+") \
                where @class = 'RGBEvent' and blob = false and timestamp between "\
                +"'"+start_time_range+ "'"+ " and " +"'"+end_time_range+"'"
        return query
get_query(0)

"select  image_base64,blob,timestamp,@class from (select expand( out( Event_in )) from Root where robot_id=1)                 where @class = 'RGBEvent' and blob = false and timestamp between '2017-12-03 16:01:52' and '2017-12-03 16:02:02'"

In [68]:
import timeit

In [72]:
for i in range(1,100):
    start_time = timeit.default_timer()
    client.command(get_query(0))
    end_time = timeit.default_timer()
    print str(end_time - start_time)

0.000446796417236
0.000380039215088
0.000550985336304
0.519398927689
0.56259894371
0.585249185562
0.599974155426
0.566572904587
0.601298809052
0.616376876831


KeyboardInterrupt: 

In [56]:
client.command("select  latitude,timestamp,@class from (select expand( out( Event_in )) from Root where robot_id=1) where @class = 'LocationEvent' and timestamp between '2017-11-15 14:07:40' and '2017-11-15 14:07:40'")

[]

In [199]:
def get_cluster_id(vertex): 
    result = client.command("select * from " + vertex)
    for res in result:
        print res
        return (res._rid.split(':')[0]).split('#')[1]
rs = get_cluster_id("LocationEvent")
if rs is None:
    

None


In [187]:
print client.command("insert into Root set robot_id=1")[0]

{'@Root':{'robot_id': 1},'version':1,'rid':'#18:1'}


In [155]:
result = client.command("select * from Root where robot_id=12")
print len(result)
for res in result:
    print res

0


In [159]:
eat_edges = client.query(
    "create edge Event_at from ("
    "select * from #18:0"
    ") to ("
    "select * from #25:0"
    ")"
)

print eat_edges[0]

PyOrientConnectionException: Server seems to have went down

In [ ]:
#step 6: create function to check vertex availability
def is_present(vertex,data):
    return True if(len(client.command("select * from "+vertex+" where "+data))>1) else False

In [27]:
### open a connection (username and password)
client.connect("root", "admin")

### create a database
client.db_create("robot", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)

### select to use that database
client.db_open("robot", "root", "admin")

### Create the Vertex Animal
client.command("create class Root extends V")

### Insert a new value
client.command("insert into Root set robot_id = 1")

### query the values
# client.query("select * from Animal")


client.command('create class Year extends V')
client.command("insert into Year set name = 2017")

client.command('create class Child extends E')
client.command('select * from Year')
### Lets the rat likes to eat pea
# eat_edges = client.command(
#     "create edge Child from ("
#     "select from Root where robot_id = 1"
#     ") to ("
#     "select from Year where name = 2017"
#     ")"
# )

# ### Who eats the peas?
# pea_eaters = client.command("select expand( in( Eat )) from Food where name = 'pea'")
# for animal in pea_eaters:
#     print(animal.name, animal.specie)
# 'rat rodent'
# ...

# ### What each animal eats?
# animal_foods = client.command("select expand( out( Eat )) from Animal")
# for food in animal_foods:
#     animal = client.query(
#                 "select name from ( select expand( in('Eat') ) from Food where name = 'pea' )"
#             )[0]
#     print(food.name, food.color, animal.name)
# 'pea green rat'

PyOrientDatabaseException: com.orientechnologies.orient.core.exception.ODatabaseException - Database named 'robot' already exists: robot
	DB name="robot"

In [148]:
def is_present(vertex,data):
    return True if(len(client.command("select * from "+vertex+" where "+data))>1) else False

In [147]:
import time
import datetime
import random

def create_time_tree(datetime,_robot_id):
    verteces = ['Root','year','Month','Day','Hour','Minute','Second','Millisecond']
    for vertex in verteces:  
        try:
            client.command("create class "+vertex+" extends V") 
        except:
            pass
    
    if (is_present(vertex,data)):
        return True
    resut = client.command("insert into "+vertex+" set " + data)
    for res in resut:
        print res._rid
create_time_tree(datetime.datetime.utcnow().strftime('%Y,%m,%d,%H,%M,%S,%f')[:-3].split(','),1)

234


True

In [121]:
def get_vertex(vertex,data):
    result = client.command("select * from "+vertex+" where "+data)
    print len(result)
get_vertex("Root","robot_id = 1")

23


In [ ]:
def check_possibility(datetime,_robot_id):
    root = Root.nodes.get_or_none(name="root",robot_id=_robot_id)
    if root is None:
        root = Root(name="root",robot_id=_robot_id).save()

    year = root.year.get_or_none(name=datetime[0])
    if year is None:
        year = Year(name=datetime[0]).save()
        root.year.connect(year)

    month = year.month.get_or_none(name=datetime[1])
    if month is None:
        month = Month(name=datetime[1]).save()
        year.month.connect(month)

    day = month.day.get_or_none(name=datetime[2])
    if day is None:
        day = Day(name=datetime[2]).save()
        month.day.connect(day)

    hour = day.hour.get_or_none(name=datetime[3])
    if hour is None:
        hour = Hour(name=datetime[3]).save()
        day.hour.connect(hour)

    minute = hour.minute.get_or_none(name=datetime[4])
    if minute is None:
        minute = Minute(name=datetime[4]).save()
        hour.minute.connect(minute)

    second = minute.second.get_or_none(name=datetime[5])
    if second is None:
        second = Second(name=datetime[5]).save()
        minute.second.connect(second)

    millisecond = second.millisecond.get_or_none(name=datetime[6])
    if millisecond is None:
        millisecond = MilliSecond(name=datetime[6]).save()
        second.millisecond.connect(millisecond)

    return millisecond

Disadvantage:
    Lack of api support for python language. Pyorient have very minimal support to access orientDB. E.g There is no API to retrive existing classes from the database.
    Cannot able to create database from pyorient, and always depend on studio protocol to create database before test run
    Cannot create vertex and edge together.

Query 3
select  longitude,latitude,timestamp,@class from (select expand( out( Event_in )) from Root where robot_id=1) where @class = 'LocationEvent' and (latitude > 30.0 and longitude < 50.0)